In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
import numpy as np
from scipy.optimize import minimize

def solve_hotelling3(prices, cost):
    n = len(prices)
    arc = 1.0 / n
    s = np.zeros(n)
    for i in range(n):
        left = (i-1)%n
        right = (i+1)%n
        s[i] = arc + (prices[right] - 2*prices[i] + prices[left])/(2*cost)
    s = np.maximum(s, 0)   # optional, for numerical safety
    s /= s.sum()
    return s

def merged_profits(phi, lambd, merged_idxs=(1,2), wtp=25, mc=None):
    phi = np.array(phi)
    n = len(phi)
    a, b = merged_idxs
    c = [i for i in range(n) if i not in merged_idxs][0]

    def make_prices(p_merged, p_rival):
        prices = np.zeros(n)
        prices[a] = prices[b] = p_merged
        prices[c] = p_rival
        return prices

    def merged_objective(p_merged, p_rival):
        if (p_merged < phi[a]) or (p_merged > wtp):
            return 10  # Penalize
        if (p_rival < phi[c]) or (p_rival > wtp):
            return 10  # (really only relevant for two-dim minimize, but harmless)
        prices = make_prices(p_merged, p_rival)
        s = solve_hotelling3(prices, lambd)
        return -((prices[a] - phi[a]) * s[a] + (prices[b] - phi[b]) * s[b])
    
    def rival_objective(p_rival, p_merged):
        if (p_rival < phi[c]) or (p_rival > wtp):
            return 10  # Penalize
        if (p_merged < phi[a]) or (p_merged > wtp):
            return 10
        prices = make_prices(p_merged, p_rival)
        s = solve_hotelling3(prices, lambd)
        return -((prices[c] - phi[c]) * s[c])

    # Initial guess
    price_eq = phi + lambd
    p_merged = price_eq[a]
    p_rival = price_eq[c]

    for _ in range(100):
        p_merged_old = p_merged
        p_rival_old = p_rival

        res_m = minimize(lambda x: merged_objective(x[0], p_rival_old), [p_merged], method='Nelder-Mead', options={'disp': False})
        p_merged = res_m.x[0]  # No need to clip, penalty ensures feasible region
        res_c = minimize(lambda x: rival_objective(x[0], p_merged_old), [p_rival], method='Nelder-Mead', options={'disp': False})
        p_rival = res_c.x[0]

        if abs(p_merged - p_merged_old) < 1e-8 and abs(p_rival - p_rival_old) < 1e-8:
            break

    prices = make_prices(p_merged, p_rival)
    s = solve_hotelling3(prices, lambd)
    insurer_profits = (prices - phi) * s
    hospital_profit = insurer_profits.sum()
    return hospital_profit, insurer_profits, s, prices

# Example usage
phi = np.array([10.95692717, 9.46563451, 9.46563451])
lambd = 6
wtp = 25
hospital_profit, insurer_profits, shares, prices = merged_profits(phi, lambd, merged_idxs=(1,2), wtp=wtp)
print("hospital profit:", hospital_profit)
print("insurer profits:", insurer_profits)
print("shares:", shares)
print("prices:", prices)

hospital profit: 3.229890867111522
insurer profits: [0.78450361 1.22269363 1.22269363]
shares: [0.36158886 0.31920557 0.31920557]
prices: [13.12652834 13.29606148 13.29606148]


In [3]:
solve_hotelling3([23,25,25],6)

array([0.66666667, 0.16666667, 0.16666667])

In [4]:
def solve_hotelling2(phi, lambd, idx):
    i, j = idx
    p_i = (2 * phi[i] + phi[j]) / 3 + lambd
    p_j = (2 * phi[j] + phi[i]) / 3 + lambd
    prices = np.zeros(len(phi))
    prices[i] = p_i
    prices[j] = p_j
    return prices


def calc_price(phi, lambd, wtp):
    phi = np.array(phi)
    n = len(phi)
    price = np.zeros(n)
    eps = 1e-8

    if n == 1:
        price = np.array([max(phi[0], min(wtp, wtp-lambd))])
    elif n == 2:
        price = solve_hotelling2(phi, lambd, [0, 1])
    elif n == 3:
        # All equal? Symmetric solution
        if np.max(phi) - np.min(phi) < eps:
            price = np.ones(n) * (phi[0] + lambd/3)
        else:
            # Try general solution
            price_try = solve_hotelling3(phi, lambd)
            ok = (price_try >= phi - eps) & (price_try <= wtp + eps)
            if np.all(ok):
                price = price_try
            else:
                # Find two lowest-cost firms, use 2-firm solution
                active = np.argsort(phi)[:2]
                price = solve_hotelling2(phi, lambd, active)
    else:
        raise NotImplementedError("Only supports n=1, 2, or 3.")
    return price




def shares_hotelling2(prices, cost, idx):
    # idx: indices of two active firms
    i, j = idx
    p_i, p_j = prices[i], prices[j]
    s_i = 0.5 + (p_j - p_i) / (2 * cost)
    s_j = 1.0 - s_i
    s = np.zeros(len(prices))
    s[i], s[j] = s_i, s_j
    return s
    
    
def calc_s(phi, cost, wtp, circle=True):
    phi = np.array(phi)
    n = len(phi)
    prices = calc_price(phi, cost, wtp)

    if np.all(prices == 0):
        return np.zeros(n)

    active = np.where(prices > 0)[0]

    if len(active) == 1:
        s = np.zeros(n)
        s[active[0]] = 1.0
        return s
    elif len(active) == 2:
        return shares_hotelling2(prices, cost, active)
    elif len(active) == 3:
        # All active; circle
        s = solve_hotelling3(prices, cost)
        # If crazy numbers (e.g., shares < 0), fallback to zeros.
        if np.any(s < 0) or np.any(s > 1):
            return np.zeros(n)
        else:
            return s
    else:
        return np.zeros(n)


def calc_profits(phi, cost, wtp, mc):
    s = calc_s(phi, cost, wtp)
    prices = calc_price(phi, cost, wtp)
    hospital_profit = np.sum(s * (phi - mc))
    insurer_profits = s * (prices - phi)
    return hospital_profit, insurer_profits, s, prices


def analytic_phi(cost, wtp, n, belief="active"):
    if n==0:
        return 0
    if n == 1:
        return (wtp - cost) / 2
    elif n == 2:
        if belief == "active":
            return 0.75*cost + (wtp-cost)/2
        elif belief == "passive":
            return 1.5*cost
    elif n == 3:
        if belief == "active":
            return cost + wtp/2
        elif belief == "passive":
            return 9/4*cost
    return None


def nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement=0, merged_idxs=None):
    # Default: use standard calc_profits
    hospital_profit, ins, s, prices = calc_profits(phi_vec, cost, wtp, mc)
    profit_idxs = [0]

    # If merging, and first agent is in the merged group, use merged profits
    if merged_idxs is not None and 0 in merged_idxs:
        hosp, ins, s, prices = merged_profits(phi_vec, cost, merged_idxs, wtp, mc)
        profit_idxs = merged_idxs
        phi_vec[profit_idxs] = phi_vec[0] #force them all equal

    ins_profit = np.sum(ins[profit_idxs])
    ins_beta = np.mean([betas[i] for i in profit_idxs])
    # Defensive: avoid log(0) or negative argument
    if hospital_profit - disagreement <= 1e-8 or ins_profit <= 1e-8 or hospital_profit <= 1e-8 :
        return 10
    obj = np.log(hospital_profit - disagreement) * (1 - ins_beta) + np.log(ins_profit) * ins_beta
    return -obj


#TODO nash objective actually depends on the thing we're optimzing over e.g. dphi1/dphi2 = 0... 
def passive_disagreement(phi_vec, cost, wtp, mc, merged_idxs=None):
    s = calc_s(phi_vec, cost, wtp)
    phi = np.array(phi_vec)
    mc = np.array(mc)
    if merged_idxs is None:
        return np.sum(s * (phi - mc)) - s[0] * (phi[0] - mc[0])

    merged_idxs = list(merged_idxs)
    other_idxs = [i for i in range(len(phi)) if i not in merged_idxs]
    return np.sum([s[i] * (phi[i] - mc[i]) for i in other_idxs])


def recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=True):
    n = len(phi_vec)
    if analytic_base and n <= 3:
        # For 1- or 2-firm base: use all analytic phi (active beliefs by default here)
        sub_phi = np.array([analytic_phi(cost, wtp, n-1, belief='active')] * n)
        hospital_profit, _, _, _ = calc_profits(sub_phi, cost, wtp, mc[:n])
        return hospital_profit
    if n == 1:
        # (should never reach here if analytic_base is True... but as fallback)
        phi_init = [analytic_phi(cost, wtp, 1, belief='active')]
        result = minimize(lambda phi: nash_obj(phi, cost, wtp, [mc[0]], [betas[0]]),
                          phi_init, method='Nelder-Mead', options={'disp': False})
        phi_star = result.x
        hosp_profit, _, _, _ = calc_profits(phi_star, cost, wtp, [mc[0]])
        return hosp_profit
    # n-1 agents: recursively solve Nash for subgame (all but the first)
    keep = list(range(1, n))
    phi_sub = np.array([phi_vec[j] for j in keep])
    mc_sub = np.array([mc[j] for j in keep])
    betas_sub = [betas[j] for j in keep]
    phi_sub_sol = solve_nash(phi_sub, cost, wtp, mc_sub, betas_sub, analytic_base=analytic_base)
    hospital_profit, _, _, _ = calc_profits(phi_sub_sol, cost, wtp, mc_sub)
    return hospital_profit
    #TODO... in case of merer... disagreement depends if facing single firm : v-lambda or merged insurer... v-lambda/2
    #TODO... basically use the logic from before " merged_idxs is not None and 0 in merged_idxs" to figure out if merged insurer
    #if  merged_idxs is not None and 0 in merged_idxs then v-lambda/2 o.w.  v-lambda

def nash_obj(phi_vec, cost, wtp, mc, betas, active=True, analytic_base=True, 
             phi_tmp_prev=0, merged_idxs=None):
    disagreement = 0
    n = len(phi_vec)
    if n == 1:
        return disagreement
    if active:
        disagreement = recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=analytic_base)
    else:
        phi_vec_tmp  = phi_vec.copy()
        phi_vec_tmp[0] = phi_tmp_prev
        disagreement = passive_disagreement(phi_vec_tmp, cost, wtp, mc, merged_idxs=merged_idxs)
        
    return nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement, merged_idxs)



#This is the work in progress one...
def solve_nash(phi_init, cost, wtp, mc, betas=None, active=False, maxiter=25, tol=1e-7, 
               analytic_base=True, merged_idxs=None):
    n = len(phi_init)
    if betas is None:
        betas = [0.5] * n
    phi = np.array(phi_init, dtype=float)
    phi_prev = phi.copy()

    # 1. Determine update indices
    if merged_idxs is not None:
        merged_idxs = sorted(list(merged_idxs))
        rep_idx = merged_idxs[0]  # Use the lowest index as "representative"
        update_indices = [rep_idx] + [i for i in range(n) if i not in merged_idxs]
    else:
        update_indices = list(range(n))

    for it in range(maxiter):
        phi_prev[:] = phi[:]  # Save old for diff check

        for k in update_indices:
            
            merged_idxs_rolled = None
            if merged_idxs is not None:
                merged_idxs_rolled = [((idx - k) % n) for idx in merged_idxs]

            
            
            def helper(phi_k_scalar):
                phi_tmp = phi_prev.copy()
                phi_tmp_prev = phi_tmp[0]
                phi_tmp[k] = phi_k_scalar[0]
                # 2a. When updating merged block, force phis in group equal (in phi_tmp)
                if merged_idxs is not None and k == rep_idx:
                    for j in merged_idxs:
                        phi_tmp[j] = phi_k_scalar[0]
                # Roll so agent k is in 0 position for nash_obj
                phi_tmp_roll = np.roll(phi_tmp, -k)
                mc_roll = np.roll(mc, -k)
                betas_roll = np.roll(betas, -k)

                if active:
                    return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll, active=True,
                                    merged_idxs=merged_idxs_rolled)
                else:
                    return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll, active=False,
                                    phi_tmp_prev=phi_tmp_prev,
                                    merged_idxs=merged_idxs_rolled)
            res = minimize(helper, [phi[k]], method='Nelder-Mead', options={'disp': False})
            phi[k] = res.x[0]
            # 2b. Synchronize all merged phis on representative updates
            if merged_idxs is not None and k == rep_idx:
                for j in merged_idxs:
                    phi[j] = phi[k]
        diff = np.max(np.abs(phi - phi_prev))
        phi_prev = phi.copy()
        print('yo 2, ', phi_prev)

    return phi


In [5]:
# --- Model run/prints ---
COST = 6
WTP = 25
MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_pass = [analytic_phi(COST, WTP, n, belief='passive')]*n

In [6]:

phi_passive = solve_nash(phi_init_pass, COST, WTP, np.zeros(n), [0.5]*n, active=False)
print("PASSIVE φ:", phi_passive)
hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit, "insurers:", insurer_profits, "shares:", s, "prices:", prices)
print("Analytic φ (passive):", analytic_phi(COST, WTP, n, "passive"))

yo 2,  [13.49983521 13.49983521 13.49983521]
yo 2,  [13.49967041 13.49967041 13.49967041]
yo 2,  [13.49950562 13.49950562 13.49950562]
yo 2,  [13.49934083 13.49934083 13.49934083]
yo 2,  [13.49917605 13.49917605 13.49917605]
yo 2,  [13.49901126 13.49901126 13.49901126]
yo 2,  [13.49884648 13.49884648 13.49884648]
yo 2,  [13.4986817 13.4986817 13.4986817]
yo 2,  [13.49851692 13.49851692 13.49851692]
yo 2,  [13.49835214 13.49835214 13.49835214]
yo 2,  [13.49818737 13.49818737 13.49818737]
yo 2,  [13.49802259 13.49802259 13.49802259]
yo 2,  [13.49785782 13.49785782 13.49785782]
yo 2,  [13.49769305 13.49769305 13.49769305]
yo 2,  [13.49752829 13.49752829 13.49752829]
yo 2,  [13.49736352 13.49736352 13.49736352]
yo 2,  [13.49719876 13.49719876 13.49719876]
yo 2,  [13.497034 13.497034 13.497034]
yo 2,  [13.49686924 13.49686924 13.49686924]
yo 2,  [13.49670448 13.49670448 13.49670448]
yo 2,  [13.49653973 13.49653973 13.49653973]
yo 2,  [13.49637498 13.49637498 13.49637498]
yo 2,  [13.49621023

In [7]:

# With a merger between firm 1 and 2
merged_idxs = (1,2)
phi_passive_merged = solve_nash(phi_init_pass, COST, WTP, MC, betas, active=False, merged_idxs=merged_idxs)
print("\nWITH MERGER (1,2):")
print("PASSIVE φ:", phi_passive_merged)

print('-----------------')
print(phi_passive_merged,
COST,
merged_idxs,
WTP,
MC
)
print('-----------------')

hospital_profit, insurer_profits, s, prices = merged_profits(phi_passive_merged,COST, merged_idxs=merged_idxs, wtp=WTP)
print("   hospital profit:", hospital_profit, "insurers:", insurer_profits, "shares:", s, "prices:", prices)

yo 2,  [10.34294128  7.82314453  7.82314453]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10.34294128  6.79253692  6.79253692]
yo 2,  [10

In [8]:
# --- Model run/prints ---
COST = 5
WTP = 26

MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_act = [analytic_phi(COST, WTP, n, belief='active')]*n
phi_active = solve_nash(phi_init_act, COST, WTP, np.zeros(n), [0.5]*n, active=True)
print("ACTIVE φ:", phi_active)
hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, COST, WTP, np.zeros(n))
print("   hospital profit:", hospital_profit_a, "insurers:", insurer_profits_a, "shares:", s_a, "prices:", prices_a)
print("Analytic φ (active):", analytic_phi(COST, WTP, n, "active"))

yo 2,  [17.99989014 17.99989014 17.99989014]
yo 2,  [17.99978027 17.99978027 17.99978027]
yo 2,  [17.99967041 17.99967041 17.99967041]
yo 2,  [17.99956055 17.99956055 17.99956055]
yo 2,  [17.99945069 17.99945069 17.99945069]
yo 2,  [17.99934083 17.99934083 17.99934083]
yo 2,  [17.99923097 17.99923097 17.99923097]
yo 2,  [17.99912111 17.99912111 17.99912111]
yo 2,  [17.99901125 17.99901125 17.99901125]
yo 2,  [17.9989014 17.9989014 17.9989014]
yo 2,  [17.99879154 17.99879154 17.99879154]
yo 2,  [17.99868168 17.99868168 17.99868168]
yo 2,  [17.99857183 17.99857183 17.99857183]
yo 2,  [17.99846198 17.99846198 17.99846198]
yo 2,  [17.99835212 17.99835212 17.99835212]
yo 2,  [17.99824227 17.99824227 17.99824227]
yo 2,  [17.99813242 17.99813242 17.99813242]
yo 2,  [17.99802256 17.99802256 17.99802256]
yo 2,  [17.99791271 17.99791271 17.99791271]
yo 2,  [17.99780286 17.99780286 17.99780286]
yo 2,  [17.99769301 17.99769301 17.99769301]
yo 2,  [17.99758316 17.99758316 17.99758316]
yo 2,  [17.99